#### 1. Import pandas library

In [2]:
import pandas as pd

#### 2. Import users table:

In [3]:
users_table=pd.read_csv("users_table.csv")

#### 3. Rename Id column to userId

In [4]:
users_table=users_table.rename(columns={"Id":"userId"})

#### 4. Import posts table:

In [5]:
posts_table=pd.read_csv("posts_table.csv")

#### 5. Rename Id column to postId and OwnerUserId to userId

In [6]:
posts_table=posts_table.rename(columns={"Id":"postId","OwnerUserId":"userId"})

#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [7]:
users=users_table[["userId","Reputation","Views","UpVotes","DownVotes"]]

posts=posts_table[["postId", "Score","userId","ViewCount","CommentCount"]]

#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [8]:
merged=pd.merge(users,posts)

#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [9]:
merged.isnull().sum()

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
ViewCount       23572
CommentCount        0
dtype: int64

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [10]:
drop_cols=merged.isnull().sum()

drop=list(drop_cols[drop_cols>0].index)

merged=merged.drop(columns=drop,axis=1)


# I think it is better to remove them. Is better to have less data but reliable, then more data with unreal views


#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [11]:
# NONE. All of the data types make sense

merged.dtypes


userId          int64
Reputation      int64
Views           int64
UpVotes         int64
DownVotes       int64
postId          int64
Score           int64
CommentCount    int64
dtype: object